In [21]:
import sqlalchemy
import pandas as pd
import chromadb
from langchain_community.vectorstores.chroma import Chroma
from tqdm.auto import tqdm

/Users/smukherjee/.pyenv/versions/3.10.14/envs/openml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
metadata_df = pd.read_csv("../../data/all_dataset_description.csv")

In [5]:
metadata_df = metadata_df.drop(columns=["Combined_information"])

In [8]:
client = chromadb.Client()

In [11]:
collec = client.get_or_create_collection("metadata")

In [20]:
client = chromadb.PersistentClient(path="test")

In [22]:
# Function to chunk the DataFrame
def chunk_dataframe(df, chunk_size):
    for i in range(0, df.shape[0], chunk_size):
        yield df.iloc[i : i + chunk_size]

In [23]:
# Define the chunk size
chunk_size = 100  # Adjust the chunk size as needed

# Process each chunk
for chunk in tqdm(
    chunk_dataframe(metadata_df, chunk_size), total=(len(metadata_df) // chunk_size) + 1
):
    ids = chunk["did"].astype(str).tolist()
    documents = chunk["description"].astype(str).tolist()
    metadatas = chunk.to_dict(orient="records")

    # Add to ChromaDB collection
    collec.add(ids=ids, documents=documents, metadatas=metadatas)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 57/57 [07:16<00:00,  7.66s/it]


In [29]:
import pandas as pd

In [31]:
pd.DataFrame.from_records(collec.get())

,data,documents,embeddings,ids,metadatas,uris
0,None,**Author**: \n**Source**: Unknown - \n**Pl...,None,10,"{'MajorityClassSize': 81.0, 'MaxNominalAttDist...",None
1,None,**Author**: \n**Source**: Unknown - Date unk...,None,1000,"{'MajorityClassSize': 3481.0, 'MaxNominalAttDi...",None
2,None,**Author**: \n**Source**: Unknown - Date unk...,None,1001,"{'MajorityClassSize': 70.0, 'MaxNominalAttDist...",None
3,None,**Author**: \n**Source**: Unknown - Date unk...,None,1002,"{'MajorityClassSize': 6694.0, 'MaxNominalAttDi...",None
4,None,**Author**: \n**Source**: Unknown - Date unk...,None,1003,"{'MajorityClassSize': 255.0, 'MaxNominalAttDis...",None
...,...,...,...,...,...,...
5685,None,**Author**: \n**Source**: Unknown - Date unk...,None,995,"{'MajorityClassSize': 1800.0, 'MaxNominalAttDi...",None
5686,None,**Author**: \n**Source**: Unknown - Date unk...,None,996,"{'MajorityClassSize': 138.0, 'MaxNominalAttDis...",None
5687,None,**Author**: \n**Source**: Unknown - Date unk...,None,997,"{'MajorityClassSize': 337.0, 'MaxNominalAttDis...",None
5688,None,**Author**: \n**Source**: Unknown - Date unk...,None,998,"{'MajorityClassSize': 33.0, 'MaxNominalAttDist...",None


In [27]:
collec.get(ids=["2", "4", "6"])

{'ids': ['2', '4', '6'],
 'embeddings': None,
 'metadatas': [{'MajorityClassSize': 684.0,
   'MaxNominalAttDistinctValues': 7.0,
   'MinorityClassSize': 8.0,
   'NumberOfClasses': 5.0,
   'NumberOfFeatures': 39.0,
   'NumberOfInstances': 898.0,
   'NumberOfInstancesWithMissingValues': 898.0,
   'NumberOfMissingValues': 22175.0,
   'NumberOfNumericFeatures': 6.0,
   'NumberOfSymbolicFeatures': 33.0,
   'Unnamed: 0': 0,
   'description': "**Author**: Unknown. Donated by David Sterling and Wray Buntine  \n\n**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/Annealing) - 1990  \n\n**Please cite**: [UCI](https://archive.ics.uci.edu/ml/citation_policy.html)  \n\n\n\nThe original Annealing dataset from UCI. The exact meaning of the features and classes is largely unknown. Annealing, in metallurgy and materials science, is a heat treatment that alters the physical and sometimes chemical properties of a material to increase its ductility and reduce its hardness, making it more workable. 